In [1]:
import tushare as ts
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import pandas as pd

def set_tushare_token(token):
    """
    Input:
        token: A string, the Tushare API token.
    Output:
        Returns a Tushare Pro API interface.
    """
    ts.set_token(token)
    return ts.pro_api()

def fetch_stock_data(tickers, start_date, end_date):
    """
    Input:
        tickers: List of stock ticker strings.
        start_date: Start date as a string in 'YYYYMMDD' format.
        end_date: End date as a string in 'YYYYMMDD' format.
    Output:
        DataFrame containing stock data for given tickers and date range.
    """
    pro = set_tushare_token('1743f38f5d63aeab1606dadd3a5a50ac6b89570155a80d8debecd01d')  
    df_stocks = pd.DataFrame()
    for ticker in tickers:
        df = pro.daily(ts_code=ticker, start_date=start_date, end_date=end_date)
        df['ts_code'] = ticker
        df_stocks = pd.concat([df_stocks, df], ignore_index=True)
    return df_stocks

def portfolio_performance(weights, mean_returns, cov_matrix, risk_free_rate=0.03):
    """
    Input:
        weights: Numpy array of portfolio weights.
        mean_returns: Series of mean returns for each stock.
        cov_matrix: DataFrame of covariance matrix of stock returns.
        risk_free_rate: Float, risk-free rate of return, default is 0.03.
    Output:
        Tuple of (returns, volatility, sharpe_ratio) for the portfolio.
    """
    returns = np.sum(mean_returns * weights)
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (returns - risk_free_rate) / volatility
    return returns, volatility, sharpe_ratio

def minimize_volatility(weights, mean_returns, cov_matrix):
    """
    Minimizes the volatility of a portfolio.
    Input:
        weights: Numpy array of portfolio weights.
        mean_returns: Series of mean returns.
        cov_matrix: DataFrame of covariance matrix.
    Output:
        Float, the calculated volatility of the portfolio.
    """
    return portfolio_performance(weights, mean_returns, cov_matrix)[1]

def optimize_portfolio(stock_data, min_return=0.001, initial_capital=100000):
    """
    Optimizes a stock portfolio to ensure a minimum return.
    Input:
        stock_data: DataFrame containing stock data.
        min_return: Float, minimum desired return for the portfolio, default is 0.001.
        initial_capital: Float, initial capital amount, default is 100000.
    Output:
        Tuple of optimized weights, expected return, expected volatility, expected sharpe ratio.
    """
    stock_data['returns'] = stock_data['close'].pct_change()
    mean_returns = stock_data.groupby('ts_code')['returns'].mean()
    cov_matrix = stock_data.pivot(index='trade_date', columns='ts_code', values='returns').cov()
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                   {'type': 'eq', 'fun': lambda x: portfolio_performance(x, *args)[0] - min_return}]
    bounds = tuple((0.0, 1.0) for asset in range(num_assets))
    initial_weights = num_assets * [1. / num_assets]
    optimized = minimize(minimize_volatility, initial_weights, args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    optimal_weights = optimized.x
    expected_return, expected_volatility, expected_sharpe = portfolio_performance(optimal_weights, *args)
    return optimal_weights, expected_return, expected_volatility

def main():
    """
    Main function to execute stock data fetching and portfolio optimization.
    Input: None
    Output: Prints the results of the portfolio optimization.
    """
    tickers = ['600519.SH', '000858.SZ', '600036.SH', '601318.SH', '000333.SZ', '600276.SH', '600030.SH', '000651.SZ', '601166.SH', '601398.SH']
    start_date = '2023-07-28'
    end_date = '2024-07-28'
    stock_data = fetch_stock_data(tickers, start_date, end_date)
    optimal_weights, expected_return, expected_volatility, = optimize_portfolio(stock_data)
    print("Optimal Weights:", optimal_weights)
    print("Expected Return:", expected_return)
    print("Expected Volatility:", expected_volatility)

if __name__ == "__main__":
    main()


Optimal Weights: [0.00000000e+00 3.84796068e-01 8.11387834e-18 0.00000000e+00
 2.17639527e-01 0.00000000e+00 1.44830877e-02 2.13384629e-18
 3.83081318e-01 0.00000000e+00]
Expected Return: 0.001000000000301619
Expected Volatility: 0.01352660563468546
